In [1]:
import pickle
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import date
from collections import namedtuple

In [2]:
import cafle as cf
from cafle.genfunc import rounding as R
from cafle.genfunc import percent as P

In [3]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"),
      pd.get_option("display.max_rows"))

40 200


# Initial Setting

In [4]:
# ctg = namedtuple('category', ['nml', 'cld'])

In [5]:
def ctg(nml, cld):
    idx = ['nml', 'cld']
    return Series([nml, cld], index = idx)

In [6]:
# Pickling Data
with open("data/area.pickle", "rb") as fr:
    data_area = pickle.load(fr)
areadf = data_area['areadf']

In [7]:
# Call out the area data
area_rent = areadf.loc[[x in data_area['rent_col'] for x in areadf.name]]
area_rent = area_rent.area_py.groupby(areadf['floor']).sum()
area_cld = round(area_rent[['b1', 'f1']].sum(), 2)
area_nml = round(area_rent[['f2', 'f3', 'f4']].sum(), 2)

### Valuation Class

In [30]:
class Valuation:
    def __init__(self, rent, mtnc, cap_rate, area, vcncy_rate, oprtg_rate, IR, RU=1_000_000):
        self.rent = rent
        self.mtnc = mtnc
        self.cap_rate = cap_rate
        self.area = area
        self.vcncy_rate = vcncy_rate
        self.oprtg_rate = oprtg_rate
        self.IR = IR
        self.RU = RU
        
        self.__intlz__()
        self.keys = ['rent', 'mtnc', 'cap_rate', 'area', 'vcncy_rate', 'oprtg_rate',
                     'PGI', 'EGI', 'oprtg_cst', 'NOI', 'value']
    
    def __intlz__(self):
        self.dpst_amt = self.rent * 10 * self.area / self.RU # deposit amount
        self.rent_amt = self.rent * 12 * self.area / self.RU # yearly rent amount
        self.mtnc_amt = self.mtnc * 12 * self.area / self.RU # yearly maintenance cost
        
        self.PGI = (self.dpst_amt * self.IR) + self.rent_amt + self.mtnc_amt
        self.EGI = self.PGI * (1 - self.vcncy_rate)
        self.oprtg_cst = self.PGI * self.oprtg_rate
        self.NOI = self.EGI - self.oprtg_cst
        self.value = self.NOI / self.cap_rate
        self.value_sum = self.value.sum()
        
    def printval(self, withsum=True):
        tmpdct = DataFrame({x: getattr(self, x) for x in self.keys})
        if withsum:
            tmpdct_withsum = tmpdct.append(DataFrame([tmpdct.sum()], index=['sum']))
            return DataFrame(tmpdct_withsum)
        else:
            return DataFrame(tmpdct)

In [53]:
class Val_Mtrx:
    def __init__(self, rent=None, mtnc=None, cap_rate=None, area=None, 
                 vcncy_rate=None, oprtg_rate=None, IR=None, RU=1_000_000):
        self.rent = rent
        self.mtnc = mtnc
        self.cap_rate=cap_rate
        self.area = area
        self.vcncy_rate = vcncy_rate
        self.oprtg_rate = oprtg_rate
        self.IR = IR
        self.RU = RU
        
        self.rslt_lst = []
        
        
    def value(self, rent=None, mtnc=None, cap_rate=None, area=None, 
                 vcncy_rate=None, oprtg_rate=None, IR=None):
        keys = ['rent', 'mtnc', 'cap_rate', 'area', 'vcncy_rate', 'oprtg_rate', 'IR']
        dct = {}
        for key in keys:
            if locals()[key] is not None:
                tmp_val = locals()[key]
            else:
                tmp_val = getattr(self, key)
            dct['tmp_'+key] = tmp_val
            
        tmpistnc = Valuation(rent = dct['tmp_rent'],
                             mtnc = dct['tmp_mtnc'],
                             cap_rate = dct['tmp_cap_rate'],
                             area = dct['tmp_area'],
                             vcncy_rate = dct['tmp_vcncy_rate'],
                             oprtg_rate = dct['tmp_oprtg_rate'],
                             IR = dct['tmp_IR'])
        return tmpistnc

    
    def mkdict(self, **kwarg):
        for key, item in kwarg.items():
            for val in item:
                tmpdct = {key:val}
                self.rslt_lst.append(self.value(**tmpdct))
                
    def mkdict_tmp(self, **kwargs):
        lenarg = len(kwargs)
        keylst = []
        arglst = []
        for key, item in kwargs.items():
            keylst.append(key)
            arglst.append(item)
        
        for val0 in arglst[0]:
            if lenarg == 1:
                tmpdct = {keylst[0]:val0}
                self.rslt_lst.append(self.value(**tmpdct))
            else:
                for val1 in arglst[1]:
                    if lenarg == 2:
                        tmpdct = {keylst[0]:val0, keylst[1]:val1}
                        self.rslt_lst.append(self.value(**tmpdct))
                    else:
                        if lenarg == 3:
                            tmpdct = {keylst[0]:val0, keylst[1]:val1, keylst[2]:val2}
                            self.rslt_lst.append(self.value(**tmpdct))
    
    def printlst(self):
        rsltdf = DataFrame()
        for df in self.rslt_lst:
            rsltdf = rsltdf.append(df.printval())
        return rsltdf

# Valuation
### 1) Assumption

In [54]:
val_mtrx = Val_Mtrx(rent = ctg(20_000, 44_000),
                    cap_rate = ctg(P(4.3), P(4.8)),
                    mtnc = ctg(2_000, 2_000),
                    area = ctg(area_nml, area_cld),
                    vcncy_rate = ctg(0.0, 0.0),
                    oprtg_rate = P(14.5),
                    IR = P(1.0))

In [55]:
rent_case = [ctg(20_000, 44_000), # unit rent
             ctg(21_000, 46_000),
             ctg(22_000, 48_000),
             ctg(23_000, 50_000)]
cap_rate = [ctg(P(4.3), P(4.8)),
            ctg(P(4.4), P(4.9)),
            ctg(P(4.5), P(5.0))]

val_mtrx.mkdict_tmp(rent=rent_case, cap_rate=cap_rate)
val_mtrx.rslt_lst

In [56]:
R(val_mtrx.rslt_lst[0].printval(withsum=True))

,rent,mtnc,cap_rate,area,vcncy_rate,oprtg_rate,PGI,EGI,oprtg_cst,NOI,value
nml,"20,000","2,000",0.0,"8,082",0.0,0.1,"2,150","2,150",312,"1,838","42,745"
cld,"44,000","2,000",0.0,"6,619",0.0,0.1,"3,683","3,683",534,"3,149","65,601"
sum,"64,000","4,000",0.1,"14,701",0.0,0.3,"5,833","5,833",846,"4,987","108,345"


In [57]:
val_mtrx.printlst()

,rent,mtnc,cap_rate,area,vcncy_rate,oprtg_rate,PGI,EGI,oprtg_cst,NOI,value
nml,20000.0,2000.0,0.043,8081.72,0.0,0.145,2149.737520,2149.737520,311.711940,1838.025580,42744.780921
cld,44000.0,2000.0,0.048,6619.06,0.0,0.145,3682.844984,3682.844984,534.012523,3148.832461,65600.676278
sum,64000.0,4000.0,0.091,14700.78,0.0,0.290,5832.582504,5832.582504,845.724463,4986.858041,108345.457198
nml,20000.0,2000.0,0.044,8081.72,0.0,0.145,2149.737520,2149.737520,311.711940,1838.025580,41773.308627
cld,44000.0,2000.0,0.049,6619.06,0.0,0.145,3682.844984,3682.844984,534.012523,3148.832461,64261.886966
sum,64000.0,4000.0,0.093,14700.78,0.0,0.290,5832.582504,5832.582504,845.724463,4986.858041,106035.195593
nml,20000.0,2000.0,0.045,8081.72,0.0,0.145,2149.737520,2149.737520,311.711940,1838.025580,40845.012880
cld,44000.0,2000.0,0.050,6619.06,0.0,0.145,3682.844984,3682.844984,534.012523,3148.832461,62976.649226
sum,64000.0,4000.0,0.095,14700.78,0.0,0.290,5832.582504,5832.582504,845.724463,4986.858041,103821.662106
nml,21000.0,2000.0,0.043,8081.72,0.0,0.145,2247.526332,2247.526332,325.891318,1921.635014,44689.186369
